### Курсовая работа Малютина Алексея

Сервисная компания, обслуживает оборудование в удаленных пунктах присутствия компании. В рамках услуги осуществляются выезды специалистов сервисной компании, доставка оборудования и его замена, которая производится со складов и точек присутствия поставщика. По договору стоимость услуги включает транспортные расходы и поставщик предоставил таблицу расстояний от своих складов до пунктов присутствия. В таблице три поля: адрес обслуживаемого подразделения, адрес склада поставщика и расстояние между двумя точками. Необходимо проверить актуальность и корректность полученных данных. Для этого необходимо независимо определить расстояние и сравнить с представленным. 

Т.к. задача разовая, однако объем данных довольно большой, то использование API сервисов карт нецелесообразно. Для решения задачи планируется осуществлить парсинг расстояний по дорогам по картам одного из сервисов, а затем сохранить полученные расстояния в датасет, чтобы сравнить с представленными. Для парсинга будем использовать Selenium with Python. 

In [28]:
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.firefox.options import Options
from selenium.common import exceptions
import time
import warnings
warnings.filterwarnings("ignore")

In [29]:
dt = pd.read_csv('distance_2_1.csv', delimiter=';')
dt.addr3 = dt.addr3.str.replace('Республика','')
dt['Distance1'] = 0

In [30]:
dt.head()

,addr1,addr3,distance,Distance1
0,"Мордовия с.Старое Шайгово, ул.Ленина, 7","Мордовия, г. Саранск, ул. Советская, 109",62.0,0
1,"Мордовия г.Инсар, ул.Московская, 91Б","Мордовия, г. Саранск, ул. Советская, 109",74.0,0
2,"Мордовия пгт.Кадошкино, ул.Крупской, 1","Мордовия, г. Саранск, ул. Советская, 109",74.0,0
3,"Мордовия с.Лямбирь, ул.Ленина, 40","Мордовия, г. Саранск, ул. Советская, 109",15.0,0
4,"Мордовия с.Ельники, площадь 1 Мая, 35","Мордовия, г. Краснослободск, ул. Интернационал...",32.0,0


In [33]:
# Преобразуем строку в вещ. число
def str_to_num(str):
    if '.' in str and str.replace('.', '').isdigit():
        return float(str)
    elif str.isdigit():
        return float(str)
    else:
        return 0.0

In [50]:
# Заполним адреса на форме и отправим ее. Полученное расстояние сохраним в датасет
def calc_distance(dt_in, delay, tbr=0):
    dt = dt_in.copy()
    t1 = time.time()
    if tbr == 0:
        browser = webdriver.Safari()
    elif tbr == 1:
        browser = webdriver.Chrome(executable_path=r"D:\Alexey\project\chromedriver.exe")
    elif tbr == 2:
        browser=webdriver.Firefox(executable_path=r"D:\Alexey\project\geckodriver.exe")
    browser.get("https://yandex.ru/maps")
    distance = browser.find_element_by_class_name("route-control")
    distance.click()
    time.sleep(2)
    count_record = 0
    count_except = 0
    for i in range(0, len(dt)):
        if dt['Distance1'][i] == 0 or dt['Distance1'][i] == np.nan: 
            try:
                field_from = browser.find_element_by_xpath("//input[@placeholder='Откуда']")
                time.sleep(delay)
                field_from.send_keys(Keys.COMMAND + "a")
                time.sleep(delay)
                field_from.send_keys(Keys.DELETE)
                time.sleep(delay)
                field_from.send_keys(dt['addr3'][i])
                time.sleep(delay)
                field_from.send_keys(Keys.DOWN)
                time.sleep(delay)
                field_from.send_keys(Keys.ENTER)
                time.sleep(delay)
                field_to = browser.find_element_by_xpath("//input[@placeholder='Куда']")
                time.sleep(delay)
                field_to.send_keys(Keys.COMMAND + "a")
                time.sleep(delay)
                field_to.send_keys(Keys.DELETE)
                time.sleep(delay)
                field_to.send_keys(dt['addr1'][i])
                time.sleep(delay)
                field_to.send_keys(Keys.DOWN)
                time.sleep(delay)
                field_to.send_keys(Keys.ENTER)
                time.sleep(delay)
                dist_km = ''
                for element in browser.find_elements_by_xpath("//*[contains(text(), 'км')]"):
                    dist_km += '' + element.text
                # print(dist_km.split('км')[1])
                rasst = dist_km.split('км')[1].replace('\xa0', '')
                rasst = rasst.replace(',','.')
                # print(rasst, end='\t')
                dt['Distance1'][i] = str_to_num(rasst)
                # print(dt['Distance1'][i])
                count_record += 1
                t2 = time.time()
                if i % 100 == 0: 
                    t = t2 - t1
                    print(f'{count_record} записей обработано')
                    print(f'{count_except} ошибок возникло')
                    print(f'Время обработки - {t:8.2f} c.')
            except (exceptions.StaleElementReferenceException, exceptions.NoSuchElementException):
                count_except += 1
                pass
    browser.close()
    t2 = time.time()
    t = t2 - t1
    print(f'Всего {count_record} записей обработано')
    print(f'Всего {count_except} ошибок возникло')
    print(f'Общее время обработки - {t:8.2f} c.')
    print('\n')
    return dt



In [51]:
# Парсим расстояния по дорогам
dt_out =  calc_distance(dt[:30], 0.5, 0)

58 
58	58
1 записей обработано
0 ошибок возникло
Время обработки -    13.76 c.
74 
74	74
74 
74	74
16 
16	16
31 
31	31
Всего 5 записей обработано
Всего 0 ошибок возникло
Общее время обработки -    44.70 c.




In [52]:
# Проверяем кол-во записей с разницей расстояний более 100 км (вероятно, ошибка расчета расстояния или парсинга)
dt_out[abs(dt_out['Distance1'].astype(float) - dt_out['distance'].astype(float)) > 100].count()

addr1        0
addr3        0
distance     0
Distance1    0
dtype: int64

In [53]:
# Устанавливаем для записей с разницей заявленного и рассчитанного растояние более 100 км значение 0
# После этого, в случае необходимости, выполняем ячейук парсинга расстояния, увеличив время между нажатиями клавиш
dt_out[abs(dt_out['Distance1'].astype(float) - dt_out['distance'].astype(float)) > 100]['Distance1'] = 0.0

In [54]:
# Повторно парсим расстояния по дорогам, увеличив время между нажатиями клавиш
dt_out =  calc_distance(dt_out[:30], 1, 0)

Всего 0 записей обработано
Всего 0 ошибок возникло
Общее время обработки -     6.31 c.




In [55]:
dt_out

,addr1,addr3,distance,Distance1
0,"Мордовия с.Старое Шайгово, ул.Ленина, 7","Мордовия, г. Саранск, ул. Советская, 109",62.0,58
1,"Мордовия г.Инсар, ул.Московская, 91Б","Мордовия, г. Саранск, ул. Советская, 109",74.0,74
2,"Мордовия пгт.Кадошкино, ул.Крупской, 1","Мордовия, г. Саранск, ул. Советская, 109",74.0,74
3,"Мордовия с.Лямбирь, ул.Ленина, 40","Мордовия, г. Саранск, ул. Советская, 109",15.0,16
4,"Мордовия с.Ельники, площадь 1 Мая, 35","Мордовия, г. Краснослободск, ул. Интернационал...",32.0,31


In [57]:
# Число записей, для которых не удалось определить расстояние
dt_out[dt_out['Distance1'] == 0].groupby(['Distance1']).count()

,addr1,addr3,distance
Distance1,,,


In [ ]:
# Находим разницу в исходном и полученном расстоянии. 
# Если разность > 0, то у поставшика услуг заявлено большее расстояние, чем полученное в ходе парсинга
dt_out['Diff'] = dt_out['distance'] - dt_out['Distance1']

In [58]:
# Создаем дополненный расстояниями файл
path = "distance_3_1.csv"
dt_out.to_csv(path)